<a href="https://colab.research.google.com/github/LennyJS/AIS2022/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project

In [6]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9add4c6cb7a230511f40c888523207a5871c3bfd5b09fc3faf1602c90e2f2395
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [7]:
import numpy as np
import pandas as pd
import glob, os
from google.colab import drive
import xml.etree.ElementTree as ET
import parser
from pandas.core.api import notnull

In [8]:
# since files are in google drive a mount is needed
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [4]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [18]:
import plotly.express as px
import gradio as gr
import matplotlib.pyplot as plt


In [9]:
df_labeled_industry = pd.read_csv("/content/gdrive/MyDrive/Firmen_df_labeled.csv", index_col = 0)

In [10]:
df_labeled_industry.head()

,ID,Name,Ort,Postleitzahl,Straße,Hausnummer,Gegenstand,Stammkapital,Gesellschaftsform,Score,Industry
0,1.0,09 Media GmbH,Regensburg,93059.0,Im Gewerbepark,A10,Anbietung von Dienstleistungen in den Bereiche...,25000.0,Gesellschaft mit beschränkter Haftung,0.960162,Advertising
1,2.0,1. Immobilienverwaltung Schülein GmbH,Wittelshofen,91749.0,Untermichelbach,50,Der Besitz und die Verwaltung von Immobilien u...,25000.0,Gesellschaft mit beschränkter Haftung,0.999847,Real Estate Operating Companies
4,5.0,10 k Transfer GmbH,Speyer,67346.0,Stuhlbrudergasse,5,Gegenstand des Unternehmens ist die Durchführu...,25000.0,Gesellschaft mit beschränkter Haftung,0.987049,Human Resource & Employment Services
6,7.0,123fahrschule Ruhrgebiet-Süd GmbH,Köln,50968.0,Klopstockstraße,1,der Betrieb einer Fahr- und Berufskraftfahrers...,25000.0,Gesellschaft mit beschränkter Haftung,0.502101,Diversified Support Services
7,8.0,123Fashiontrade.com GmbH,Neuss,41460.0,"Breslauer Straße 1, Haus Milano Raum",205,"Großhandel mit Textilien, Spielwaren und Acces...",25000.0,Gesellschaft mit beschränkter Haftung,0.890654,Trading Companies & Distributors


In [11]:
unique_industries = df_labeled_industry["Industry"].unique()
print(unique_industries)
print(len(unique_industries))

['Advertising' 'Real Estate Operating Companies'
 'Human Resource & Employment Services' 'Diversified Support Services'
 'Trading Companies & Distributors' 'Asset Management & Custody Banks'
 'Commodity Chemicals' 'Research & Consulting Services'
 'Personal Products' 'Leisure Products' 'Movies & Entertainment'
 'Apparel, Accessories & Luxury Goods' 'Property & Casualty Insurance'
 'Investment Banking & Brokerage' 'Technology Distributors' 'Homebuilding'
 'Casinos & Gaming' 'Specialty Chemicals' 'IT Consulting & Other Services'
 'Specialty Stores' 'Health Care Facilities'
 'Electrical Components & Equipment' 'Construction & Engineering'
 'Internet & Direct Marketing Retail' 'Industrial Machinery'
 'Systems Software' 'Building Products' 'Aerospace & Defense'
 'Auto Parts & Equipment' 'Environmental & Facilities Services' 'Steel'
 'Restaurants' 'Health Care Services' 'Oil & Gas Refining & Marketing'
 'Consumer Finance' 'Internet Services & Infrastructure'
 'Electric Utilities' 'Packaged F

In [12]:
unique_industries = unique_industries.tolist()

In [21]:
company_names = df_labeled_industry["Name"].unique()
company_names = company_names.tolist() 

In [23]:
print(company_names)

['09 Media GmbH', '1. Immobilienverwaltung Schülein GmbH', '10 k Transfer GmbH', '123fahrschule Ruhrgebiet-Süd GmbH', '123Fashiontrade.com GmbH', '140Fahrenheit GmbH', '190 Holding GmbH', '1A Media Union GmbH', '1A SAFE GmbH', '1Body GmbH', '1gn!tE GmbH', '1KFZ UG (haftungsbeschränkt)', '2. Immo-Invest GmbH', '2 K Digital Repro GmbH Studio für kreative Druckvorstufe', '2-Rad-Center Schröder GmbH', '2b-objective UG (haftungsbeschränkt)', '2birds GmbH', '2HR GmbH', '3. Annika & Annabelle Spreeufer Grundstücksgesellschaft mbH', '3. HBW Grundstücks UG (haftungsbeschränkt)', '360° Versicherungen GmbH', '395 Ventures GmbH', '3B Gesellschaft für Hoteldienstleistungen mbH', '3F-Energy GmbH', '3H Elektro + Sanitär Haustechnik, Handel und Holding Aktiengesellschaft', '3H - International Change Management GmbH', '3P Projects GmbH', '3Plan AG', '3Stil-Bau GmbH', '4 people Gesellschaft für Werbung mbH', '4. Spieloase Casilino GmbH', '42INVEST GmbH', '4artis GmbH', '4Athletes Sports GmbH', '4grands 

In [22]:
len(company_names)

7184

# Names of business leaders

In [29]:
df_names = pd.read_csv("/content/gdrive/MyDrive/Personen_Df.csv", index_col = 0)
df_names.head()

,ID,Vorname,Nachname,Ort,Geburtsdatum,Beruf
0,1,Michael,Simmendinger,Riekofen,1968-07-21,NaN
1,2,Richard,Schülein,Wittelshofen,1976-08-13,NaN
2,3,Thomas,Lohmann,Köln,1964-11-30,NaN
3,4,Mike,Klocke-Müller,NaN,1979-06-13,NaN
4,5,Kerstin,Pöhl-Sievertsen,NaN,1972-07-10,NaN


In [40]:
def combine_cols(row):
    return str(row['Vorname']) + ' ' + row['Nachname']

In [41]:
df_names['Full Name'] = df_names.apply(combine_cols, axis=1)

In [42]:
print(df_names.head())

   ID  Vorname         Nachname           Ort Geburtsdatum Beruf  \
0   1  Michael     Simmendinger      Riekofen   1968-07-21   NaN   
1   2  Richard         Schülein  Wittelshofen   1976-08-13   NaN   
2   3   Thomas          Lohmann          Köln   1964-11-30   NaN   
3   4     Mike    Klocke-Müller           NaN   1979-06-13   NaN   
4   5  Kerstin  Pöhl-Sievertsen           NaN   1972-07-10   NaN   

                 Full Name  
0     Michael Simmendinger  
1         Richard Schülein  
2           Thomas Lohmann  
3       Mike Klocke-Müller  
4  Kerstin Pöhl-Sievertsen  


# Industry names for Dropdown list


In [14]:
def Dropdown_list(df_labeled_industry):
    return df_labeled_industry['Industry']

In [15]:
def plot_barchart(df_labeled_industry):
    fig = plt.figure()
    plt.boxplot(x=df_labeled_industry['Industry']) #, y=df_pens['bill_depth_mm'])
    return fig

# Creation of App with Gradio

In [26]:
demo = gr.Blocks()

options_industry = unique_industries
options_persons = 
options_company = company_names 

with demo:
  Dropdown_Industry=gr.inputs.Dropdown(options_industry, label='Enter an industry'),
  #Personen=gr.inputs.Text()
  Company_names=gr.inputs.Textbox(company_names, label='Enter the company name you are looking for'),
  fn=plot_barchart,
  title="Barchart depicting the different industries and the number of associated companies",
  description="Check for the different industries"
  
demo.launch(debug=False)

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>